In [ ]:
import basic
import geopandas as gpd
import pandas as pd
import numpy as np
import flopy
import pathlib
import make_wells

In [ ]:
print('running wel creation package')
datestart = '6/19/2012'

numdays = (pd.to_datetime('10/1/2022')-pd.to_datetime(datestart)).days

df = make_wells.load_caissons()
# df = make_wells.get_period(df, datestart,numdays, False)
# df = df.droplevel(1,0)
# df = df/43560.



In [ ]:
df.loc[:,:].groupby(df.index.year).transform(lambda x: x.cumsum()).replace({0:np.nan}).plot(subplots = True)

In [ ]:
df.plot(subplots = True)

In [ ]:
df.loc[:,:].groupby(df.index.to_period('M').to_timestamp('M')).transform(lambda x: x.fillna(x.mean()))

In [ ]:
df = df.resample("1D").mean()
df = df.reindex(index = pd.date_range(datestart, periods = numdays, freq = 'D'))
df = df.bfill().ffill()

c = df.sum(axis=1) == 0

df.loc[c,:] = df.mean().mean()

In [ ]:
df = df.resample("1D").mean()

In [ ]:
import hydro_context


In [ ]:
hydro_context.run_current_context('June2012')

In [ ]:
hydro_context.run_long_context()

In [ ]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr
import time as tm

# Set time to download data from (this is always the 00UTC run of the present day)
time_year = str(tm.localtime()[0])
time_month = str(tm.localtime()[1])
time_day = str(tm.localtime()[2]-1)

if len(time_month)== 1:
    time_month = '0' + time_month
datestr = time_year + time_month + time_day
print('The run chosen is the 00 UTC run of ' + time_day + '-' + time_month + '-' + time_year)

# Define server information
serverstring='http://nomads.ncep.noaa.gov:9090/dods/gens_bc/gens' + datestr + '/gep_all_00z'
print(serverstring)

# Load data 
dataset = xr.open_dataset(serverstring)
time = dataset.variables['time']  
lat = dataset.variables['lat'][:]
lon = dataset.variables['lon'][:]
lev = dataset.variables['lev'][:]
ens = dataset.variables['ens'][:]

# Select user settings to plot (in this case all timesteps for all (20) members for a box around the Netherlands near the surface)
time_toplot = time  # select all available timesteps
lat_toplot = np.arange(50, 55, 0.5)
lon_toplot = np.arange(2, 8, 0.5)
lev_toplot = np.array([1000])
ens_toplot = ens  # select all available ensemble members

# Select required data via xarray
dataset = dataset.sel(ens=ens_toplot, time=time_toplot, lev=lev_toplot, lon=lon_toplot, lat=lat_toplot)

# Loading the data into memory finally results in the error
u = dataset.variables["ugrdprs"].values

In [ ]:
import hydro_context

In [ ]:
hydro_context.plot_total_by_season(df)

In [ ]:
for r in range(2012, 2021):
    run = f"June{r}"
    basic.setup_folder(run)
    # make_wells.run(run)

In [ ]:

df = make_wells.run('June2012')

In [ ]:
df = df.loc['5/28/2017':'6/15/2017']

# df = df.droplevel(1,0)

In [ ]:
df

In [ ]:
df.loc['5/29/2017':'6/10/2017',:] = np.nan

In [ ]:
df.index = df.index+pd.to_timedelta(1, unit = 'H')

In [ ]:
df.index = df.index.date

In [ ]:
df

In [ ]:
df.resample('1D').sum()

In [ ]:
df

In [ ]:
np.rec.array([tuple(x) for x in df.values.tolist()   ],
             dtype = [('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('stage', '<f4'),])

In [ ]:


np.rec.array([tuple(df.values.tolist()[0]),  ],
             dtype = [('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('stage', '<f4'),])

In [ ]:
df = pd.DataFrame(np.random.random((10,4)), columns = ['k', 'i', 'j', 'stage'])
df.loc[:,'k':'j'] = 1
df = df.astype({'k':int, 'i':int, 'j':int})
np.rec.array(df.values.tolist(),dtype = [('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('stage', '<f4'),])
# df.to_records(index = False, column_dtypes=riv_dtype)

In [ ]:
flopy.modflow.ModflowWel.get_default_dtype()

In [ ]:
riv_dtype = flopy.modflow.ModflowWel.get_default_dtype()
riv_dtype

In [ ]:
ml = basic.load_model()

In [ ]:
# ml.wel.stress_period_data[0]

In [ ]:
# ml.wel.export('GIS/wells.shp' )

In [ ]:
# wel = ml.wel.stress_period_data[0]

# wel = pd.DataFrame(wel)

# wel.loc[:,'row'] = wel.loc[:,'i'] +1

# wel.loc[:,'column'] = wel.loc[:,'j'] +1

In [ ]:
# grid = gpd.read_file("GIS/grid.shp")

In [ ]:
# welflux = pd.merge(grid, wel, on = ['row', 'column'])

# welflux = gpd.GeoDataFrame(welflux, crs = 2226)

In [ ]:
wells = gpd.read_file('GIS/wells.shp')

In [ ]:
wells.loc[wells.loc[:,'flux']==0,'flux'] = -1 # some wells have no flow info, so setting to 1.

In [ ]:
smf = wells.groupby('wellname').sum().loc[:,'flux'].rename('sumflux')
wells = pd.merge(wells, smf, on = 'wellname')


wells.loc[:,'frac'] = wells.loc[:,'flux']/wells.loc[:,'sumflux']

In [ ]:
wells.head(1)

In [ ]:
path = pathlib.Path("T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons - AvailableDailyAverages\DailyData\Caissons")
    


In [ ]:
def loadcaisson(path, caisson = 'Caisson1Flow.csv', name = 'Well1'):
    c = pd.read_csv(path.joinpath(caisson))
    c = c.set_index(pd.to_datetime(c.loc[:,'DateTime'])).loc[:,['Value']].rename(columns = {'Value':name})
    c = c.resample('1D').sum().applymap(basic.isnumber)
    
    return c

def load_pumps(path, name, pumps):
    p1 = loadcaisson(path,caisson = f'Pump{pumps[0]}Flow.csv', name = name)
    p2 = loadcaisson(path,caisson = f'Pump{pumps[1]}Flow.csv', name = name)
    
    p1 = p1.applymap(basic.isnumber)
    p2 = p2.applymap(basic.isnumber)
    
    ptot = p1+p2
    
    return ptot
    
    
def load_caissons():
    
    '''
    get well pumping in feet^3/day
    '''
    path = pathlib.Path(r"T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons - AvailableDailyAverages\DailyData\Caissons")
    
    c1 = loadcaisson(path, caisson = 'Caisson1Flow.csv', name = "well1")
    c2 = loadcaisson(path, caisson = 'Caisson2Flow.csv', name = "well2")
    c6 = loadcaisson(path, caisson = 'Caisson6Flow.csv', name = "well6")

    c3 = load_pumps(path, 'well3', [5,6])
    c4 = load_pumps(path, 'well4', [7,8])
    c5 = load_pumps(path, 'well5', [9,10])
    
    df = pd.concat([c1, c2, c3, c4, c5, c6], axis = 1).fillna(0.)
    
    df = df.astype({ic:np.float64 for ic in df.columns})
    
    df[df.abs().values>1e10] = 0
    
    df = df*1e6 / 7.480543 #from MGD to feet^3/day
    
    return df

def get_period(df, start_date, numdays):


    df = df.loc[start_date:, :].iloc[:numdays]

    # df.loc[:, 'Value'] = df.loc[:, 'Value'].interpolate()

    assert df.isnull().sum().sum() == 0, 'has nans'
    assert df.index.to_series().diff().nunique()==1, 'index has missing days'
    
    df.loc[:,'per'] = np.arange(numdays)
    df = df.set_index('per', append = True)
    
    return df

def get_well_info(wells, timeseries):
    
    wells_info = pd.merge(wells, df.droplevel(0,0).T.stack().to_frame('Q').reset_index(), 
                          left_on = 'wellname', right_on = 'level_0').sort_values('per')
    
    wells_info.loc[:,'qcell'] = -1 * wells_info.loc[:,'Q'].abs() * wells_info.loc[:,'frac'] / (24*60*60)
    
    return wells_info
    
def plot_pumping(df, out_folder):
    ax = df.droplevel(1,0).rename(lambda x: pd.to_datetime(x).strftime("%b\n%d\n%Y")).mul(1/43560).plot.bar(
        stacked = True, figsize = (10,10), ylabel = 'acre-feet', grid = True, title = "Total Caisson Pumping, per Well")

    ax.set_xticks(ax.get_xticks()[::7])
    ax.tick_params(axis="x", rotation=0)
    
    plt.savefig(os.path.join(out_folder, 'pumping.png'), dpi=250, bbox_inches = 'tight')
    
def mf_wel(m, ts_data):
    
    stress_period_data = {}

    for per, group in ts_data.loc[:,['per','k','i', 'j', 'qcell']].groupby('per'):
        stress_period_data[per] = group.drop(columns = 'per').values

    wel = flopy.modflow.ModflowWel(m, ipakcb = 1,filenames ='RR.wel',
                               stress_period_data = stress_period_data)
    
    return wel
    
def run(name):
    print('running wel creation package')
    m = basic.load_model(name)

    info, swr_info, sfr_info, riv_keys_info = basic.load_params(name)

    datestart = info['start_date']
    numdays = info['numdays']

    out_folder = basic.out_folder(name)
    df = load_caissons()

    df = get_period(df, datestart,numdays)

    ts_data = get_well_info(wells, df)

    wel = mf_wel(m, ts_data)

    wel.write_file()

    plot_pumping(df, out_folder)
    
run('June2013')

In [ ]:

c = [('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('stage', '<f4'), ('cond', '<f4'), ('rbot', '<f4')]
dict(c)

In [ ]:
riv_dtype.

In [ ]:

    

riv_dtype = flopy.modflow.ModflowRiv.get_default_dtype()
print(riv_dtype)

[('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('stage', '<f4'), ('cond', '<f4'), ('rbot', '<f4')]

# Now that we know the structure of the recarray that we want to create, we can create a new one as follows.

stress_period_data = np.zeros((1,10), dtype=riv_dtype)
print(stress_period_data)
print(stress_period_data.dtype)
print(stress_period_data.shape)

In [ ]:
stress_period_data = stress_period_data.view(np.recarray)
print("stress_period_data: ", stress_period_data)
print("type is: ", type( ))

In [ ]:
df = pd.DataFrame(np.random.random((10,6)))
df

In [ ]:
df.to_records(False)

In [ ]:
df.values.view(  riv_dtype)  

In [ ]:
stress_period_data[0] = df.values.view(np.recarray)                                       

In [ ]:
stress_period_data[0]

In [ ]:
stress_period_data[1]

In [ ]:
ax.get_xticklabels()[0]

In [ ]:
df.loc['2021'].plot.bar(stacked = True, figsize = (10,10))

In [ ]:
df.resample('1Q').sum().describe()

In [ ]:
df.resample('1Q').sum().plot.bar(stacked = True, figsize = (20,10))

In [ ]:
df.resample('1Q').sum().sum(axis=1).idxmax()

In [ ]:
df.idxmax()

In [ ]:
df.loc[df.idxmax()]

In [ ]:
df.index.to_series().diff().unique()

In [ ]:
df.index.to_series().diff().nunique()